In [4]:
import sys
import numpy as np
import pandas as pd
from IPython.display import display

sys.path.append("../")
from vpop_calibration.model.ode import OdeModel
from vpop_calibration.model.structural_model import StructuralOdeModel
from vpop_calibration.nlme import NlmeModel
from vpop_calibration.saem import PySAEM
from vpop_calibration.data_generation import simulate_dataset_from_omega

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
def equations(t, y, k_a, k_12, k_21, k_el):
    # y[0] is A_absorption, y[1] is A_central, y[2] is A_peripheral
    A_absorption, A_central, A_peripheral = y[0], y[1], y[2]
    dA_absorption_dt = -k_a * A_absorption
    dA_central_dt = (
        k_a * A_absorption + k_21 * A_peripheral - k_12 * A_central - k_el * A_central
    )
    dA_peripheral_dt = k_12 * A_central - k_21 * A_peripheral

    ydot = [dA_absorption_dt, dA_central_dt, dA_peripheral_dt]
    return ydot


variable_names = ["A0", "A1", "A2"]
parameter_names = ["k_a", "k_12", "k_21", "k_el"]

pk_two_compartments_abs_model = OdeModel(equations, variable_names, parameter_names)

In [12]:
# Create an NLME model
# First the structural model
# We need an ODE model, initial conditions and time steps

ode_model = pk_two_compartments_abs_model
print(ode_model.variable_names)

protocol_design = pd.DataFrame(
    {"protocol_arm": ["arm-A", "arm-B"], "k_el": [0.5, 5.2]}
)  # this is just a dummy design
time_span = (0, 24)
nb_steps = 20

# For each output and for each patient, give a list of time steps to be simulated
time_steps = np.linspace(time_span[0], time_span[1], nb_steps).tolist()

initial_conditions = np.array([10.0, 0.0, 0.0])

# Parameter definitions
true_log_MI = {"k_a": 0.5}
true_log_PDU = {
    "k_12": {"mean": -1, "sd": 0.25},
    "k_21": {"mean": 0, "sd": 0.25},
}
error_model_type = "additive"
true_res_var = [0.05, 0.02, 0.1]
true_covariate_map = {
    "k_12": {
        "foo": {"coef": "cov_foo_k12", "value": 0.5},
        "bar": {"coef": "cov_bar_k12", "value": 1.5},
    },
    "k_21": {},
}  # list for each PDU, which covariate influences it, and the name of the coefficient

patient_covariates = pd.DataFrame(
    {
        "id": ["a", "b", "c", "d"],
        "protocol_arm": ["arm-A", "arm-A", "arm-B", "arm-B"],
        "foo": [1, 2, 3, 4],
        "bar": [1, 1, 1, 1],
    }
)

['A0', 'A1', 'A2']


In [13]:
obs_df = simulate_dataset_from_omega(
    ode_model,
    protocol_design,
    time_steps,
    initial_conditions,
    true_log_MI,
    true_log_PDU,
    error_model_type,
    true_res_var,
    true_covariate_map,
    patient_covariates,
)

display(obs_df)

Successfully loaded 2 covariates:
['bar', 'foo']


,k_a,k_21,k_12,id,protocol_arm,foo,bar,output_name,time,A0_0,A1_0,A2_0,k_el,value
0,3.172386,1.445735,1.648721,a,arm-A,1,1,A0,0.000000,10.0,0.0,0.0,0.5,10.000000
1,3.172386,1.445735,1.648721,a,arm-A,1,1,A0,1.263158,10.0,0.0,0.0,0.5,0.181838
2,3.172386,1.445735,1.648721,a,arm-A,1,1,A0,2.526316,10.0,0.0,0.0,0.5,0.003307
3,3.172386,1.445735,1.648721,a,arm-A,1,1,A0,3.789474,10.0,0.0,0.0,0.5,0.000060
4,3.172386,1.445735,1.648721,a,arm-A,1,1,A0,5.052632,10.0,0.0,0.0,0.5,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,9.754583,0.700769,1.648721,d,arm-B,4,1,A2,18.947368,10.0,0.0,0.0,5.2,0.000146
236,9.754583,0.700769,1.648721,d,arm-B,4,1,A2,20.210526,10.0,0.0,0.0,5.2,0.000077
237,9.754583,0.700769,1.648721,d,arm-B,4,1,A2,21.473684,10.0,0.0,0.0,5.2,0.000040
238,9.754583,0.700769,1.648721,d,arm-B,4,1,A2,22.736842,10.0,0.0,0.0,5.2,0.000021


In [14]:
struct_model = StructuralOdeModel(
    pk_two_compartments_abs_model, protocol_design, initial_conditions
)

In [21]:
init_log_MI = {"k_a": 0.5}
init_log_PDU = {
    "k_12": {"mean": -1, "sd": 0.25},
    "k_21": {"mean": 0, "sd": 0.25},
}
init_res_var = [0.05, 0.02, 0.1]
init_covariate_map = {
    "k_12": {
        "foo": {"coef": "cov_foo_k12", "value": -0.5},
        "bar": {"coef": "cov_bar_k12", "value": 0.0},
    },
    "k_21": {},
}

nlme_model = NlmeModel(
    struct_model,
    patient_covariates,
    init_log_MI,
    init_log_PDU,
    init_res_var,
    init_covariate_map,
    error_model_type,
)

Successfully loaded 2 covariates:
['bar', 'foo']


In [22]:
optimizer = PySAEM(
    nlme_model,
    obs_df,
    mcmc_burn_in=1,
    mcmc_first_burn_in=1,
    mcmc_nb_samples=1,
    mcmc_proposal_var_scaling_factor=0.5,
    nb_phase1_iterations=10,
    nb_phase2_iterations=None,
    convergence_threshold=1e-4,
    patience=5,
    learning_rate_power=0.8,
    annealing_factor=0.95,
    verbose=False,
)

In [23]:
optimizer.run()

  5%|▌         | 1/20 [00:19<06:02, 19.07s/it]


KeyboardInterrupt: 